# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    # Write your code here
    # Not guaranteed to be 2 securities in the portfolio
    total_invested = 0
    for row in P.index:
        total_invested += P.iloc[row]['Amount Invested']
    
    beta_portfolio = 0
    for row in P.index:
        beta_portfolio += P.iloc[row]['Beta'] * (P.iloc[row]['Amount Invested']/total_invested)
    
    return rf + mkt_rp * beta_portfolio

In [4]:
expected_return(portfolio, 0.02, 0.05)

0.086

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [5]:
def volatility(P, cor):
    # Write your code here
    # Using the Portfolio Volatility with Arbitrary Weights formulas
    variance = 0
    total_invested = 0
    for row in P.index:
        total_invested += P.iloc[row]['Amount Invested']
    
    for i in P.index:
        for j in P.index:
            if i == j:
                variance += (P.iloc[i]['Amount Invested']/total_invested)**2 * (P.iloc[i]['Volatility'])**2
            else:
                variance += (P.iloc[i]['Amount Invested']/total_invested) * (P.iloc[i]['Volatility']) * (P.iloc[j]['Amount Invested']/total_invested) * (P.iloc[j]['Volatility']) * cor
            
    return variance**0.5

In [6]:
volatility(portfolio, -0.2)

0.24083189157584592

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [7]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    # Write your code here
    weight =  vol/mkt_vol
    
    return rf + mkt_rp * weight


In [8]:
expected_return2(0.02, 0.05, 0.16, 0.2408)

0.09525

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as $Cov(R_i-R_f, R_{mkt} – R_f)/Var(R_{mkt}-R_f)$ $\rightarrow$ $\frac{Cov(R_i-R_f, R_{mkt} – R_f)}{Var(R_{mkt}-R_f)}$. 

expected return: it should return output of 3225 rows × 3 columns

In [6]:
def find_return(df, fp):
    excess_ret = []
    mkt_rf = []
    
    # append the R_i - R_f and R_{mkt} - R_f to df
    for row in df.index:
        excess_ret.append(df.iloc[row]['RET'] - (fp.loc[fp['date'] == df.iloc[row]['date'], 'RF'].iloc[0]))
        mkt_rf.append(fp.loc[fp['date'] == df.iloc[row]['date'], 'Mkt-RF'].iloc[0])
    df['ri-rf'] = excess_ret
    df['mkt-rf'] = mkt_rf
    
    return df
    
def CAPM_beta(return_path, factors_path):
    #Write your code here
    df = pd.read_csv(return_path)
    df['date'] = df['year'] * 100 + df['month']
    
    fp = pd.read_csv(factors_path)
    fp['Mkt-RF'] = fp['Mkt-RF']/100
    fp['RF'] = fp['RF']/100
    
    df = find_return(df, fp)
    
    groups = df.groupby(["CUSIP"], group_keys = False)
    cusip = []
    exp_ret = []
    beta = []
    
    for name, group in groups:
        # Get CUSIP
        cusip.append(name)
        
        # Get the Exp_return
        avg_ret = np.mean(group["RET"])
        exp_ret.append(avg_ret)
        
        # Find covariance and variance
        cov = np.cov(group['ri-rf'], group['mkt-rf'], ddof=0)[0, 1]
        var = group['mkt-rf'].var(ddof=0)
        
        # Append beta
        if (var == 0):
            beta.append(np.nan)
        else:
            beta.append(cov/var)
        
    output = pd.DataFrame(data = {
        "CUSIP" : cusip,
        "EXP_RETURN" : exp_ret,
        "BETA" : beta,
    })
    
    return output

In [10]:
return_path = '../../datafiles/A7/monthlycrsp_mod7.csv'
factors_path = '../../datafiles/A7/factors.CSV'

# Takes a while (~10 min)
data = CAPM_beta(return_path, factors_path)
data

CUSIP  EXP_RETURN       BETA
0  (36020,)   -0.032023  -0.483703
1  (36021,)    0.028259  -1.124363
2  (36022,)    0.048830  -5.149374
3  (36023,)    0.089881  10.187062

CUSIP  EXP_RETURN      BETA
0     00081T10    0.016252  2.591073
1     00108M10    0.011101  0.741057
2     00130H10    0.016152  1.457100
3     00163U10    0.015538  0.975530
4     00182C10    0.014772  1.406085
...        ...         ...       ...
3220   9959910    0.009933  0.763107
3221   9972410    0.012866  1.280548
3222   9972510    0.017177  1.065298
3223   9984910    0.009006  1.876418
3224   9985510    0.015699  1.166459

[3225 rows x 3 columns]

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [3]:
def slope(data):
    coefficients = np.polyfit(data['BETA'], data['EXP_RETURN'], 1)

    slope_value = coefficients[1]
    
     # This should return 0.007981588120535642... with the provided data.
    return slope_value

In [4]:
test_beta_frame = pd.DataFrame(data = {
    "CUSIP" : [36020,36021,36022,36023],
    "EXP_RETURN" : [0.03,0.05,0.07,0.08],
    "BETA": [-0.3,1.2, 4.6, 0.23]
})

slope(test_beta_frame)

0.05139683205850835